In [ ]:
# Box plot (distribution #comment)

In [ ]:
import pyes
import json
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from bokeh.charts import BoxPlot,Bar, show

def NumofPostPerMonth(fids,host,pngFilePath):
    data = []
    for fid in fids:
        conn = pyes.es.ES(host)
        tq = pyes.query.TermQuery(field="fid", value=fid)
        tagg = pyes.aggs.TermsAgg('pid', field= 'fid_pid')
        qsearch = pyes.query.Search(tq) 
        qsearch.agg.add(tagg) 
        result = conn.search(query=qsearch , indices='facebook_nested' , doc_types='comment')
        for fidpid_summary in result.aggs["pid"]["buckets"]:
            if fidpid_summary["doc_count"] < outlierFilter or outlierFilter < 0:
                data.append([fidpid_summary["doc_count"],fid,fidpid_summary["key"]])

    
        df=pd.DataFrame(data,columns={'count','fidpid','fid'})
    print df
####figure
    p = BoxPlot(df, values='count', label='fid', legend=None,  title='Distribution of # of comments')
    show(p)

    
host = '127.0.0.1:9200'
outlierFilter=5000

fid = ["154929428406",'100489613340306','115361891807579']

NumofPostPerMonth(fid,host,outlierFilter)

In [ ]:
#post per month for specified page

In [ ]:
import pyes
import json
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from bokeh.charts import BoxPlot,Bar, show

def barNumofPostPerMonth(fid,host,lowerbound,upperbound):
    conn = pyes.es.ES(host)
    tq = pyes.query.TermQuery(field="fid", value=fid)
    DHAgg = pyes.aggs.DateHistogramAgg('month' ,field='created_time', interval='month',sub_aggs=[])
    tagg = pyes.aggs.TermsAgg('pid', field= 'fid_pid')
    DHAgg.sub_aggs.append(tagg) 
    qsearch = pyes.query.Search(tq) 
    qsearch.agg.add(DHAgg) 
    result = conn.search(query=qsearch , indices='facebook_nested' , doc_types='comment')
    i=0
    data=[]
    for r in result.aggs["month"]["buckets"]:
        monthyear=datetime.datetime.strptime(r["key_as_string"], "%Y-%m-%dT%H:%M:%S.%fZ").strftime('%Y-%m')
        data.append([len(r['pid']['buckets']),monthyear])  
        
    
    df=pd.DataFrame(data,columns={'count','date'}) 
    print data
####figure
    p = Bar(df, values='count', label='date', legend=None,
        xlabel='Month', ylabel='# of post', title=fid+'-Monthly post')
    show(p)

    
host = '127.0.0.1:9200'
lower_bound = 0
upper_bound = 500

fid1 = '154929428406'     
fid2 = '100489613340306'  
fid3 = '115361891807579'

barNumofPostPerMonth(fid1,host,lower_bound,upper_bound)

In [ ]:
#likes_count

In [ ]:
import pyes
import json
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from bokeh.charts import BoxPlot,Bar, show

host = '127.0.0.1:9200'

fid1 = '154929428406'     #1922
fid2 = '100489613340306'  #Samsung Mobile Taiwan
fid3 = '115361891807579'  #Domino's Pizza Taiwan

fid = ["154929428406",'100489613340306','115361891807579']

def bb(fids):
    data=[]
    for fid in fids:
        conn = pyes.es.ES(host)
        tq = pyes.query.TermQuery(field="fid", value=fid)
        qsearch = pyes.query.Search(tq)  
        result = conn.search(query=qsearch , indices='facebook_nested' , doc_types='fanpage')

        
        data.append([fid,result[0]['likes_count']])

    df=pd.DataFrame(data,columns={'fid','likes_count'})
    print df
    p = Bar(df, values='likes_count', label='fid', legend=None,
    xlabel='Fid', ylabel='# of likes', title='Likes of Fanpages')
    show(p)

bb(fid)

In [ ]:
# # of comment - PostTime

In [ ]:
import pyes
import json
import numpy as np
import pandas as pd
import datetime
from bokeh.charts import BoxPlot,Bar, show

def AGG(fids):
    data = []
    for fid in fids:
        conn = pyes.es.ES(host)
        tq = pyes.query.TermQuery(field="fid", value=fid)
        #tagg = pyes.aggs.TermsAgg('mes', field= 'message')
        qsearch = pyes.query.Search(tq) 
        #qsearch.agg.add(tagg) 
        result = conn.search(query=qsearch , indices='facebook_nested' , doc_types='post')
    
        for i in range(len(result)):
            T = datetime.datetime.fromtimestamp(int(str(result[i]["created_at_linux_timestamp"])[:-3])).strftime('%w')
            #print T
            data.append([T,result[i]["comment_count"],fid])


    df=pd.DataFrame(data,columns={'comment_count','date','group'})

    p = Bar(df, values='comment_count', label='date', group='group',
    xlabel='Date', ylabel='# of comment', legend='top_right',title='# of comment-PostTime(week)')
    show(p)

host = '127.0.0.1:9200'

fid = ["154929428406",'100489613340306','115361891807579']

AGG(fid)